# Parameter Sweep Analysis

This notebook analyzes intervention results across different parameter configurations to identify the optimal setup.

**Parameters varied:**
- `top_k_heads`: [5, 10, 20]
- `intervention_strength`: [2.0, 5.0, 10.0, 100.0]

**Total configurations:** 3 × 4 = 12

## 1. Setup & Configuration

In [ ]:
import pickle
import json
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
from itertools import product
import warnings
warnings.filterwarnings('ignore')

# Set plotting style
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (14, 8)
plt.rcParams['font.size'] = 10

print("Libraries imported successfully!")

In [ ]:
# ============================================================================
# CONFIGURATION - Modify these to match your experiment
# ============================================================================

RUN_ID = "20251120_120007"  # Your experiment run ID
PROBE_TYPE = "attention"  # attention, mlp, or both
RESULTS_DIR = Path("robust_experiment_results/intervention_results")

# Parameter grid (should match your bash script)
TOP_K_LIST = [5, 10, 20]
INTERVENTION_STRENGTHS = [2.0, 5.0, 10.0, 100.0]
DEMOGRAPHICS = ['gender', 'race', 'ideology']  # Order matters!

print(f"Configuration:")
print(f"  Run ID: {RUN_ID}")
print(f"  Probe Type: {PROBE_TYPE}")
print(f"  Results Directory: {RESULTS_DIR}")
print(f"  Top-K values: {TOP_K_LIST}")
print(f"  Intervention strengths: {INTERVENTION_STRENGTHS}")
print(f"  Demographics: {DEMOGRAPHICS}")
print(f"  Total configurations: {len(TOP_K_LIST) * len(INTERVENTION_STRENGTHS)}")

## 2. Load Data

In [ ]:
# Load all result files
all_results = []
missing_files = []

for demographic in DEMOGRAPHICS:
    for top_k, strength in product(TOP_K_LIST, INTERVENTION_STRENGTHS):
        # Construct filename with k and s parameters
        filename = f"{demographic}_{PROBE_TYPE}_k{top_k}_s{strength}_{RUN_ID}_intervention_results.pkl"
        filepath = RESULTS_DIR / filename
        
        if filepath.exists():
            with open(filepath, 'rb') as f:
                data = pickle.load(f)
            
            # Extract overall metrics
            metrics = data['overall_metrics']
            
            result = {
                'demographic': demographic,
                'top_k': top_k,
                'strength': strength,
                'baseline_acc_mean': metrics['baseline_accuracy_mean'],
                'baseline_acc_std': metrics.get('baseline_accuracy_std'),
                'intervention_acc_mean': metrics['intervention_accuracy_mean'],
                'intervention_acc_std': metrics.get('intervention_accuracy_std'),
                'improvement_mean': metrics['improvement_mean'],
                'improvement_std': metrics.get('improvement_std'),
                'kendall_tau_baseline': metrics.get('baseline_kendall_tau_mean'),
                'kendall_tau_intervention': metrics.get('intervention_kendall_tau_mean'),
                'kendall_improvement': metrics.get('kendall_improvement_mean'),
                'macro_f1_baseline': metrics.get('baseline_macro_f1_mean'),
                'macro_f1_intervention': metrics.get('intervention_macro_f1_mean'),
                'macro_f1_improvement': metrics.get('macro_f1_improvement_mean'),
                'cohen_kappa_intervention': metrics.get('intervention_cohen_kappa_mean'),
                'n_folds': data.get('n_folds', 3)
            }
            all_results.append(result)
        else:
            missing_files.append(filename)

# Create DataFrame
df = pd.DataFrame(all_results)

print(f"\n{'='*80}")
print(f"DATA LOADING SUMMARY")
print(f"{'='*80}")
print(f"Successfully loaded: {len(all_results)} configurations")
print(f"Missing files: {len(missing_files)}")

if missing_files:
    print(f"\nMissing configurations:")
    for f in missing_files[:5]:  # Show first 5
        print(f"  - {f}")
    if len(missing_files) > 5:
        print(f"  ... and {len(missing_files) - 5} more")

print(f"\nLoaded data shape: {df.shape}")
display(df.head(10))

## 3. Overall Best Configuration

In [ ]:
# Calculate mean improvement across all demographics for each configuration
config_performance = df.groupby(['top_k', 'strength']).agg({
    'improvement_mean': ['mean', 'std', 'count'],
    'intervention_acc_mean': 'mean',
    'baseline_acc_mean': 'mean'
}).reset_index()

# Flatten column names
config_performance.columns = ['top_k', 'strength', 'improvement_mean', 'improvement_std', 
                              'n_demographics', 'intervention_acc', 'baseline_acc']

# Sort by improvement
config_performance = config_performance.sort_values('improvement_mean', ascending=False)

print("\n" + "="*80)
print("TOP 5 CONFIGURATIONS (by mean improvement across demographics)")
print("="*80)
display(config_performance.head())

# Announce winner
best = config_performance.iloc[0]
print(f"\n🏆 BEST OVERALL CONFIGURATION:")
print(f"   top_k_heads = {int(best['top_k'])}")
print(f"   intervention_strength = {best['strength']}")
print(f"   Mean improvement: {best['improvement_mean']:+.2f} points")
print(f"   Std across demographics: ±{best['improvement_std']:.2f}")
print(f"   Baseline accuracy: {best['baseline_acc']*100:.1f}%")
print(f"   Intervention accuracy: {best['intervention_acc']*100:.1f}%")

## 4. Heatmap Visualizations

In [ ]:
# Create heatmaps for each demographic
fig, axes = plt.subplots(1, len(DEMOGRAPHICS), figsize=(18, 5))

for idx, demographic in enumerate(DEMOGRAPHICS):
    ax = axes[idx] if len(DEMOGRAPHICS) > 1 else axes
    
    # Filter data for this demographic
    demo_df = df[df['demographic'] == demographic]
    
    # Create pivot table: top_k (rows) × strength (columns)
    heatmap_data = demo_df.pivot(index='top_k', columns='strength', values='improvement_mean')
    
    # Plot heatmap
    sns.heatmap(heatmap_data, annot=True, fmt='.2f', cmap='RdYlGn', center=0,
                ax=ax, cbar_kws={'label': 'Improvement (points)'},
                vmin=-5, vmax=5)  # Adjust range as needed
    
    ax.set_title(f'{demographic.capitalize()} Intervention', fontsize=12, fontweight='bold')
    ax.set_xlabel('Intervention Strength', fontsize=10)
    ax.set_ylabel('Top-K Heads', fontsize=10)
    ax.invert_yaxis()  # Higher k at top

plt.suptitle('Accuracy Improvement by Configuration and Demographic', 
             fontsize=14, fontweight='bold', y=1.02)
plt.tight_layout()
plt.show()

# Find best configuration for each demographic
print("\n" + "="*80)
print("BEST CONFIGURATION PER DEMOGRAPHIC")
print("="*80)
for demographic in DEMOGRAPHICS:
    demo_df = df[df['demographic'] == demographic]
    best_idx = demo_df['improvement_mean'].idxmax()
    best_row = demo_df.loc[best_idx]
    print(f"\n{demographic.capitalize()}:")
    print(f"  top_k = {int(best_row['top_k'])}, strength = {best_row['strength']}")
    print(f"  Improvement: {best_row['improvement_mean']:+.2f} ± {best_row['improvement_std']:.2f} points")

## 5. Parameter Sensitivity Analysis

In [ ]:
# Plot 1: Effect of top_k (separate lines for each strength)
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 6))

# Left plot: Improvement vs top_k
for strength in INTERVENTION_STRENGTHS:
    data = df.groupby(['top_k', 'strength'])['improvement_mean'].mean().reset_index()
    strength_data = data[data['strength'] == strength]
    ax1.plot(strength_data['top_k'], strength_data['improvement_mean'], 
             marker='o', label=f'strength={strength}', linewidth=2)

ax1.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax1.set_xlabel('Top-K Heads', fontsize=11)
ax1.set_ylabel('Mean Improvement (points)', fontsize=11)
ax1.set_title('Effect of Top-K on Improvement', fontsize=12, fontweight='bold')
ax1.legend(title='Intervention Strength')
ax1.grid(alpha=0.3)

# Right plot: Improvement vs strength
for top_k in TOP_K_LIST:
    data = df.groupby(['top_k', 'strength'])['improvement_mean'].mean().reset_index()
    topk_data = data[data['top_k'] == top_k]
    ax2.plot(topk_data['strength'], topk_data['improvement_mean'], 
             marker='o', label=f'top_k={top_k}', linewidth=2)

ax2.axhline(y=0, color='red', linestyle='--', linewidth=1, alpha=0.5)
ax2.set_xlabel('Intervention Strength', fontsize=11)
ax2.set_ylabel('Mean Improvement (points)', fontsize=11)
ax2.set_title('Effect of Strength on Improvement', fontsize=12, fontweight='bold')
ax2.set_xscale('log')  # Log scale for strength
ax2.legend(title='Top-K Heads')
ax2.grid(alpha=0.3)

plt.tight_layout()
plt.show()

# Statistical summary
print("\n" + "="*80)
print("PARAMETER SENSITIVITY SUMMARY")
print("="*80)

# Effect of top_k (averaged over all strengths)
topk_effect = df.groupby('top_k')['improvement_mean'].agg(['mean', 'std'])
print("\nEffect of Top-K (averaged across all strengths):")
display(topk_effect)

# Effect of strength (averaged over all top_k)
strength_effect = df.groupby('strength')['improvement_mean'].agg(['mean', 'std'])
print("\nEffect of Strength (averaged across all top_k):")
display(strength_effect)

## 6. Advanced Metrics Comparison

In [ ]:
# Compare top 3 configurations across multiple metrics
top_configs = config_performance.head(3)

fig, axes = plt.subplots(2, 2, figsize=(14, 10))
axes = axes.flatten()

metrics = [
    ('improvement_mean', 'Accuracy Improvement (points)'),
    ('kendall_improvement', "Kendall's Tau Improvement"),
    ('macro_f1_improvement', 'Macro F1 Improvement'),
    ('cohen_kappa_intervention', "Cohen's Kappa (Intervention)")
]

for idx, (metric, label) in enumerate(metrics):
    ax = axes[idx]
    
    # Get data for top 3 configs across all demographics
    plot_data = []
    labels = []
    
    for _, config in top_configs.iterrows():
        config_data = df[(df['top_k'] == config['top_k']) & 
                        (df['strength'] == config['strength'])]
        
        values = config_data[metric].dropna()
        if len(values) > 0:
            plot_data.append(values.tolist())
            labels.append(f"k={int(config['top_k'])}, s={config['strength']}")
    
    if plot_data:
        bp = ax.boxplot(plot_data, labels=labels, patch_artist=True)
        for patch in bp['boxes']:
            patch.set_facecolor('lightblue')
        
        ax.set_ylabel(label, fontsize=10)
        ax.set_xlabel('Configuration', fontsize=10)
        ax.set_title(f'Top 3 Configs: {label}', fontsize=11, fontweight='bold')
        ax.grid(axis='y', alpha=0.3)
        
        # Rotate x labels
        ax.tick_params(axis='x', rotation=15)
    else:
        ax.text(0.5, 0.5, f'No data for {label}', 
                ha='center', va='center', transform=ax.transAxes)

plt.suptitle('Multi-Metric Comparison of Top 3 Configurations', 
             fontsize=14, fontweight='bold')
plt.tight_layout()
plt.show()

## 7. Robustness Analysis

In [ ]:
# Analyze variance across folds (robustness)
# Lower std = more robust/consistent results

# Calculate mean improvement vs std for each configuration
config_robustness = df.groupby(['top_k', 'strength']).agg({
    'improvement_mean': 'mean',
    'improvement_std': 'mean'  # Average std across demographics
}).reset_index()

# Scatter plot: improvement vs robustness
fig, ax = plt.subplots(figsize=(12, 8))

scatter = ax.scatter(config_robustness['improvement_mean'], 
                     config_robustness['improvement_std'],
                     s=200, alpha=0.6, c=config_robustness['top_k'], 
                     cmap='viridis', edgecolors='black', linewidth=1.5)

# Annotate each point
for _, row in config_robustness.iterrows():
    ax.annotate(f"k={int(row['top_k'])}\ns={row['strength']}", 
                (row['improvement_mean'], row['improvement_std']),
                fontsize=8, ha='center', va='center')

ax.axvline(x=0, color='red', linestyle='--', linewidth=1, alpha=0.5, label='Zero improvement')
ax.set_xlabel('Mean Improvement (points)', fontsize=11)
ax.set_ylabel('Mean Std Dev (across folds)', fontsize=11)
ax.set_title('Performance vs Robustness Trade-off', fontsize=13, fontweight='bold')
ax.grid(alpha=0.3)

# Add colorbar
cbar = plt.colorbar(scatter, ax=ax)
cbar.set_label('Top-K Heads', fontsize=10)

# Ideal region (high improvement, low std)
ax.axhspan(0, config_robustness['improvement_std'].quantile(0.25), 
           alpha=0.1, color='green', label='Low variance region')
ax.legend()

plt.tight_layout()
plt.show()

# Find most robust configurations
robust_configs = config_robustness.nsmallest(5, 'improvement_std')
print("\n" + "="*80)
print("MOST ROBUST CONFIGURATIONS (lowest variance across folds)")
print("="*80)
display(robust_configs)

# Best trade-off: high improvement AND low std
# Normalize both metrics to [0, 1] and find best combined score
config_robustness['norm_improvement'] = (
    (config_robustness['improvement_mean'] - config_robustness['improvement_mean'].min()) /
    (config_robustness['improvement_mean'].max() - config_robustness['improvement_mean'].min())
)
config_robustness['norm_std'] = (
    (config_robustness['improvement_std'] - config_robustness['improvement_std'].min()) /
    (config_robustness['improvement_std'].max() - config_robustness['improvement_std'].min())
)
config_robustness['combined_score'] = config_robustness['norm_improvement'] - config_robustness['norm_std']

best_tradeoff = config_robustness.nlargest(3, 'combined_score')
print("\n" + "="*80)
print("BEST PERFORMANCE-ROBUSTNESS TRADE-OFF")
print("="*80)
display(best_tradeoff[['top_k', 'strength', 'improvement_mean', 'improvement_std', 'combined_score']])

## 8. Recommendations

In [ ]:
print("\n" + "="*80)
print("FINAL RECOMMENDATIONS")
print("="*80)

# 1. Overall best
best = config_performance.iloc[0]
print(f"\n1. BEST OVERALL CONFIGURATION:")
print(f"   → top_k_heads = {int(best['top_k'])}")
print(f"   → intervention_strength = {best['strength']}")
print(f"   → Expected improvement: {best['improvement_mean']:+.2f} ± {best['improvement_std']:.2f} points")

# 2. Most robust
most_robust = robust_configs.iloc[0]
print(f"\n2. MOST ROBUST CONFIGURATION (lowest variance):")
print(f"   → top_k_heads = {int(most_robust['top_k'])}")
print(f"   → intervention_strength = {most_robust['strength']}")
print(f"   → Improvement: {most_robust['improvement_mean']:+.2f} ± {most_robust['improvement_std']:.2f} points")

# 3. Best trade-off
tradeoff = best_tradeoff.iloc[0]
print(f"\n3. BEST PERFORMANCE-ROBUSTNESS TRADE-OFF:")
print(f"   → top_k_heads = {int(tradeoff['top_k'])}")
print(f"   → intervention_strength = {tradeoff['strength']}")
print(f"   → Improvement: {tradeoff['improvement_mean']:+.2f} ± {tradeoff['improvement_std']:.2f} points")

# 4. Per-demographic recommendations
print(f"\n4. PER-DEMOGRAPHIC RECOMMENDATIONS:")
for demographic in DEMOGRAPHICS:
    demo_df = df[df['demographic'] == demographic]
    best_idx = demo_df['improvement_mean'].idxmax()
    best_row = demo_df.loc[best_idx]
    print(f"   {demographic.capitalize()}: k={int(best_row['top_k'])}, s={best_row['strength']} "
          f"({best_row['improvement_mean']:+.2f} points)")

# 5. Suggested next experiments
print(f"\n5. SUGGESTED NEXT EXPERIMENTS:")

# Check if best is at boundary
best_k = int(best['top_k'])
best_s = best['strength']

if best_k == min(TOP_K_LIST):
    print(f"   - Try lower top_k values (e.g., 3, 4) - best was at minimum tested")
elif best_k == max(TOP_K_LIST):
    print(f"   - Try higher top_k values (e.g., 30, 50) - best was at maximum tested")
else:
    print(f"   - top_k seems optimal around {best_k}")

if best_s == min(INTERVENTION_STRENGTHS):
    print(f"   - Try lower strength values (e.g., 0.5, 1.0) - best was at minimum tested")
elif best_s == max(INTERVENTION_STRENGTHS):
    print(f"   - Try higher strength values (e.g., 200, 500) - best was at maximum tested")
else:
    print(f"   - strength seems optimal around {best_s}")

print(f"   - Fine-tune around best config: k ∈ [{max(5, best_k-2)}, {best_k+2}], "
      f"s ∈ [{best_s*0.5}, {best_s*2}]")

print("\n" + "="*80)

## 9. Export Best Configuration

In [ ]:
# Export best configuration to JSON for easy reuse
best_config = {
    'top_k_heads': int(best['top_k']),
    'intervention_strength': float(best['strength']),
    'expected_improvement': float(best['improvement_mean']),
    'improvement_std': float(best['improvement_std']),
    'intervention_accuracy': float(best['intervention_acc']),
    'baseline_accuracy': float(best['baseline_acc']),
    'run_id': RUN_ID,
    'probe_type': PROBE_TYPE,
    'demographics': DEMOGRAPHICS
}

# Uncomment to save
# import json
# with open(f'best_config_{RUN_ID}.json', 'w') as f:
#     json.dump(best_config, f, indent=2)
# print(f"Best configuration saved to best_config_{RUN_ID}.json")

print("\nBest Configuration:")
print(json.dumps(best_config, indent=2))